# Watermass transformation in the CDW density range



Watermass transformation rates are defined as:

\begin{eqnarray}
    \Omega &=& \frac{\partial}{\partial \sigma} \int \int \int \frac{D \sigma'}{D t} dV, \\
    \Omega(\sigma, t)_{i,j} &=& A_c \frac{1}{\Delta \sigma} \sum^{N_z} \bigg (  \frac{D \sigma}{D t}  h_c \Delta z_f \delta(\sigma - \sigma') \bigg ),
\end{eqnarray}

$$ \frac{D \sigma}{Dt} = \frac{\partial \sigma}{\partial \theta} \dot{\theta} + \frac{\partial \sigma}{\partial S}\dot{S} $$

$$\dot{\theta} = \frac{D \theta}{Dt} = G^\theta_{hdiff} + G^\theta_{vdiff} + G^\theta_{surf} + G^\theta_{SW}$$

$$\dot{S} = \frac{D S}{Dt} = G^S_{hdiff} + G^S_{vdiff} + G^S_{surf}$$


Since we have the theta and salinity advective terms available as online outputs at monthly frequency, we can use that to directly obtain the $\dot{\theta} = \frac{\partial \theta}{\partial t} + $ and $\dot{S}$, where 


In [1]:
%config Completer.use_jedi = False

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import cmocean as cm
import xarray as xr
import numpy as np
import IPython.display
import cosima_cookbook as cc
import pandas as pd


In [3]:
session = cc.database.create_session()

In [3]:
expList = cc.querying.get_experiments(session)

In [99]:
expList.loc[expList["experiment"].str.contains("01deg_jra55v140_iaf_cycle4")]

,experiment,ncfiles
94,01deg_jra55v140_iaf_cycle4,131990
156,01deg_jra55v140_iaf_cycle4_jra55v150_extension,13422
184,01deg_jra55v140_iaf_cycle4_rerun_from_1983,172
185,01deg_jra55v140_iaf_cycle4_rerun_from_1986,86
188,01deg_jra55v140_iaf_cycle4_rerun_from_1980,2582


In [4]:
experiment = "01deg_jra55v140_iaf_cycle4"

In [101]:
fileList = cc.querying.get_ncfiles(session, experiment)

In [5]:
varList = cc.querying.get_variables(session, experiment=experiment, frequency="1 monthly")

In [91]:
varList["frequency"].unique()

array([None, '1 daily', '1 monthly', 'static'], dtype=object)

In [11]:
varList.loc[varList["name"].str.lower().str.contains("ice")]#.loc[163, "ncfile"]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
15,aice_m,ice area (aggregate),1,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
16,aicen_m,"ice area, categories",1,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
109,sfc_salt_flux_ice,sfc_salt_flux_ice,kg/(m^2*sec),1 monthly,output991/ocean/ocean-2d-sfc_salt_flux_ice-1-m...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
113,snoice_m,snow-ice formation,cm/day,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
156,vicen_m,"ice volume, categories",m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00


In [ ]:
average_DT = cc.querying.getvar(experiment, "average_DT", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [12]:
u = cc.querying.getvar(experiment, "u", session, frequency="1 monthly")

0.3.0


In [14]:
v = cc.querying.getvar(experiment, "v", session, frequency="1 monthly")

In [62]:
temp = cc.querying.getvar(experiment, "temp", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [136]:
temp = temp.sel(yt_ocean = slice(-90, -60))

In [63]:
temp_xflux_adv = cc.querying.getvar(experiment, "temp_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [137]:
temp_xflux_adv = temp_xflux_adv.sel(yt_ocean = slice(-90, -60))

In [64]:
temp_yflux_adv = cc.querying.getvar(experiment, "temp_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [138]:
temp_yflux_adv = temp_yflux_adv.sel(yu_ocean = slice(-90, -60))

In [ ]:
salt = cc.querying.getvar(experiment, "salt", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt = salt.sel(yt_ocean = slice(-90, -60))

salt_xflux_adv = cc.querying.getvar(experiment, "salt_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt_xflux_adv = salt_xflux_adv.sel(yt_ocean = slice(-90, -60))

salt_yflux_adv = cc.querying.getvar(experiment, "salt_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt_yflux_adv = salt_yflux_adv.sel(yt_ocean = slice(-90, -60))

In [139]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5})
                         })

In [140]:
delta_lat_t2t = np.diff(ds["yt_ocean"])
delta_lat_t2t = np.append(delta_lat_t2t, delta_lat_t2t[-1])

delta_lat_u2u = np.diff(ds["yu_ocean"].values)
delta_lat_u2u = np.append(delta_lat_u2u, delta_lat_u2u[-1])

In [141]:
Re = 6370e3 # Radius of the earth in meters

In [142]:
delta_y_t2t = Re * np.deg2rad(delta_lat_t2t) # delta(lat) converted to distance in meters 
delta_y_u2u = Re * np.deg2rad(delta_lat_u2u) # delta(lat) converted to distance in meters 

In [143]:
delta_lon_t2t = np.diff(ds["xt_ocean"])
delta_lon_t2t = np.append(delta_lon_t2t, delta_lon_t2t[-1])

In [144]:
delta_lon_u2u = np.diff(ds["xu_ocean"])
delta_lon_u2u = np.append(delta_lon_u2u, delta_lon_u2u[-1])

In [145]:
delta_x_t2t = np.zeros((ds["yt_ocean"].shape[-1], ds["xu_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_t2t[i] = Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_t2t)

In [148]:
delta_x_u2u = np.zeros((ds["yt_ocean"].shape[-1], ds["xt_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_u2u[i] = Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_u2u)

In [157]:
time_vars = temp.time

In [158]:
time_vars

<xarray.DataArray 'time' (time: 351)>
array(['1989-10-16T12:00:00.000000000', '1989-11-16T00:00:00.000000000',
       '1989-12-16T12:00:00.000000000', ..., '2018-10-16T12:00:00.000000000',
       '2018-11-16T00:00:00.000000000', '2018-12-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1989-10-16T12:00:00 ... 2018-12-16T12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   GREGORIAN
    bounds:          time_bounds

In [149]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5}),
                          "delta_y_t2t":(["yu_ocean"], delta_y_t2t, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "delta_y_u2u":(["yt_ocean"], delta_y_u2u, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "delta_x_t2t":(["yt_ocean", "xu_ocean"], delta_x_t2t),
                          "delta_x_u2u":(["yt_ocean", "xt_ocean"], delta_x_u2u),
                         })

In [153]:
from xgcm import Grid

In [154]:
xgrid = Grid(ds, 
            metrics={("X",):["delta_x_t2t", "delta_x_u2u"],
              ("Y",):["delta_y_t2t", "delta_y_u2u"]
            })

In [155]:
xgrid.derivative(temp_xflux_adv, "X")

<xarray.DataArray (time: 351, st_ocean: 75, yt_ocean: 490, xt_ocean: 3600)>
dask.array<truediv, shape=(351, 75, 490, 3600), dtype=float64, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -60.15 -60.1 -60.05
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
Dimensions without coordinates: time, st_ocean

In [156]:
xgrid.derivative(temp_yflux_adv, "Y")

<xarray.DataArray (time: 351, st_ocean: 75, yt_ocean: 490, xt_ocean: 3600)>
dask.array<truediv, shape=(351, 75, 490, 3600), dtype=float64, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -60.15 -60.1 -60.05
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
Dimensions without coordinates: time, st_ocean

In [ ]:
volume_cell_t = ds["delta_x_u2u"] * ds["delta_y_u2u"] * ds["delta_s_w2w"]

In [ ]:
ts, te = "1990-01-01", "1990-06-01" # time start and end


Dtemp_Dt = time_derivative(temp) + xgrid.diff(temp_xflux_adv.sel(time = slice(ts, te)), "X") / (volume_cell_t*Cp*rho0) + \
           xgrid.diff(temp_yflux_adv.sel(time = slice(ts, te)), "Y") / (volume_cell_t*Cp*rho0)

In [ ]:
ts, te = "1990-01-01", "1990-06-01" # time start and end


Dsalt_Dt = time_derivative(salt) + xgrid.derivative(salt_xflux_adv.sel(time = slice(ts, te)), "X") / (rho0) + \
           xgrid.derivative(temp_yflux_adv.sel(time = slice(ts, te)), "Y") / (rho0)